In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import math
import io
from PIL import Image
from copy import deepcopy
from IPython.display import HTML
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")
# device='cpu'
from torch.utils.data import Dataset, DataLoader
import random
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
import numpy as np
from matplotlib.colors import Normalize
import random
from torch.utils.data import DataLoader, random_split, Subset
import cv2
print(device)

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# this is used to create few samples from training and saved them to apply data augmentation technique to them and create synthetic data

# path='/home/scai/mtech/aib222677/scratch/Task2/data'
# data_path_Train = os.path.join(path,'train')
# img_path=[os.path.join(data_path_Train, filename) for filename in os.listdir(data_path_Train)]
# original_dataset=[]
# for i,data in enumerate(img_path):
#     image = Image.open(data)
#     to_tensor = transforms.ToTensor()
#     image=to_tensor(image)
#     if i%5==0:
#         original_dataset.append(image)



In [ ]:
augmentation_transform = transforms.Compose([
    transforms.RandomRotation(45),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

augmentation_transform_gaussian = transforms.Compose([
   
    transforms.GaussianBlur(kernel_size=29, sigma=11),
    
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

augmentation_transform_gamma= transforms.Compose([transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

     

In [ ]:

path='/home/scai/mtech/aib222677/scratch/Task2/data'
path_test='/home/scai/mtech/aib222677/scratch/Task4/data'
path1='/home/scai/mtech/aib222677/scratch/Task4/C/test'
data_path_Train = os.path.join(path,'train') #Enter the train folder directory
data_path_Test = os.path.join(path_test,'test') #Enter the test folder directory
batch_size = 30
num_workers = 2 
transform_train_x = transforms.Compose([
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
     # for data augmentation
                                # transforms.GaussianBlur(kernel_size=5),
                               # transforms.RandomRotation(degrees=60),
                                     ])
                               
transform_train_y = transforms.Compose([
                                
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                       # for data augmentation
                                # transforms.GaussianBlur(kernel_size=5),
                               # transforms.RandomRotation(degrees=60),
                               
                                     ])

transform= transforms.Compose([transforms.Resize((256,512)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [ ]:

# this will create a class for training data containing both mri sequences 9real and target) as a single merge image of shape (256,512)
class MRIImage(Dataset):
    def __init__(self, data_dir, transform1=None,transform2=None):
        self.data_dir = data_dir
        self.transform1 = transform1
        self.transform2=transform2
        # self.transform3=transform3
        self.image_paths = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]

        

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        # image=
        to_tensor = transforms.ToTensor()
        image=to_tensor(image)
        # print(image.shape)
        img1=image[:,:,:256]
        img2=image[:,:,256:]

        # if random.random() < 0.1:
        if self.transform1:
            # if random.random()<0.1:
            #     img1=transforms.functional.adjust_gamma(img1,gamma=0.5,gain=1) # gamma data augmentation
            # else:
            #     img1=img1
        # else:
            img1= self.transform1(img1)
            # if random.random()<0.1:
            #     img1=transforms.functional.adjust_gamma(img1,gamma=0.8,gain=1)
            # else:
            #     img1=img1
        # else:
        #     img1= self.transform3(img1)
            
        if self.transform2:
            img2=self.transform2(img2)
            
        image=torch.cat((img1, img2), dim=2)
        
        return image

In [ ]:
# this will create a class for test data containing both mri sequences and mask of them

class MRIImageTestMasked(Dataset):
    def __init__(self, data_dir1,transform=None):
        self.data_dir1 = data_dir1
       
        self.transform = transform
        # path to input and output merge image

        self.image_paths1 = [os.path.join(data_dir1, filename) for filename in sorted(os.listdir(data_dir1)) if os.path.isfile(os.path.join(data_dir1, filename))]
       # path to mask image
        self.image_paths2=[x.replace('data','C') for x in self.image_paths1]
       
    def __len__(self):
        # Return the length of the dataset based on the number of images in the directories
        return min(len(self.image_paths1), len(self.image_paths2))

    def __getitem__(self, idx):
        # Load images from the respective directories based on the index
        image_path1 = self.image_paths1[idx]
        image_path2 = self.image_paths2[idx]

        image1 = Image.open(image_path1)
        image2 = Image.open(image_path2)

        # Apply transformations if specified
        if self.transform:
            image1 = self.transform(image1) # merge image of input and target of size(256,512)
            
        image2_array = np.array(image2) # mask image

        # Return a dictionary containing the two images
        return {'image1': image1, 'image2': image2_array}


In [ ]:
# class to create synthetic data using augmentation technique
# class AugmentedImage(Dataset):
#     def __init__(self, data,transform1=None,transform2=None,transform3=None):
#         self.data = data
#         self.transform1 =transform1
#         self.transform2=transform2
#         self.transform3=transform3

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, index):
#         image= self.data[index]


#         random_number = random.randint(1,2)
    
#         if random_number == 1:                           (rotation)
#            augmented_image = self.transform1(image)
#            augmented_dataset.append(augmented_image)

            
        
#         elif random_number == 2:          (gamma transform)
#            img1=image[:,:,:256]
#            img2=image[:,:,256:]
#            img1=transforms.functional.adjust_gamma(img1,gamma=0.5,gain=1)
#            i1=self.transform3(img1)
#            i2=self.transform3(img2)
#            img=torch.cat((i1, i2), dim=2)
#            # augmented_dataset.append(img)
#            image=img
            
#         else:                     (gaussian blur)
#             img1=image[:,:,:256]
#             img2=image[:,:,256:]
#             augmented_image = self.transform2(img1)
#             img=torch.cat((augmented_image, img2), dim=2)
#             image=img
           


#         return image

# augmented_data=AugmentedImage(original_dataset,augmentation_transform,augmentation_transform_gaussian,augmentation_transform_gamma)



In [ ]:

# Load the entire training dataset
full_dataset =MRIImage(data_dir=data_path_Train, transform1=transform_train_x,transform2=transform_train_y)
# print(len(full_dataset))   #338 patient data for training



train_dataset=full_dataset
# when adding synthetic data to the training
# train_dataset=full_dataset+augmented_data

# print(len(train_dataset))
# Create a data loader for training
load_Train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [ ]:
# Load the entire test dataset
full_test_dataset = MRIImageTestMasked(data_dir1=data_path_Test,transform=transform)

test_dataset=full_test_dataset
# print(len(test_dataset))
# Create a data loader for testing
load_Test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
#function used to split input and target
def split(img):
    return img[:,:,:,:256], img[:,:,:,256:]

In [ ]:
inst_norm = True if batch_size==1 else False  # instance normalization


def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    return nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride,
    padding=padding)


def conv_n(in_channels, out_channels, kernel_size, stride=1, padding=0, inst_norm=False):
    if inst_norm == True:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding), nn.InstanceNorm2d(out_channels,
        momentum=0.1, eps=1e-5),)
    else:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding), nn.BatchNorm2d(out_channels,
        momentum=0.1, eps=1e-5),)

def tconv(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0,):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride,
    padding=padding, output_padding=output_padding)

def tconv_n(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, inst_norm=False):
    if inst_norm == True:
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding, output_padding=output_padding),
        nn.InstanceNorm2d(out_channels, momentum=0.1, eps=1e-5),)
    else:
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding, output_padding=output_padding),
        nn.BatchNorm2d(out_channels, momentum=0.1, eps=1e-5),)

In [ ]:
dim_c = 3
dim_g = 64

# Generator class designed for taking input of shape (256,256,3)
class Gen(nn.Module):
    def __init__(self, inst_norm=False):
        super(Gen,self).__init__()
        self.n1 = conv(dim_c, dim_g, 4, 2, 1)
        self.n2 = conv_n(dim_g, dim_g*2, 4, 2, 1, inst_norm=inst_norm)
        self.n3 = conv_n(dim_g*2, dim_g*4, 4, 2, 1, inst_norm=inst_norm)
        self.n4 = conv_n(dim_g*4, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n5 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n6 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n7 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n8 = conv(dim_g*8, dim_g*8, 4, 2, 1)

        self.m1 = tconv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m2 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m3 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m4 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m5 = tconv_n(dim_g*8*2, dim_g*4, 4, 2, 1, inst_norm=inst_norm)
        self.m6 = tconv_n(dim_g*4*2, dim_g*2, 4, 2, 1, inst_norm=inst_norm)
        self.m7 = tconv_n(dim_g*2*2, dim_g*1, 4, 2, 1, inst_norm=inst_norm)
        self.m8 = tconv(dim_g*1*2, dim_c, 4, 2, 1)
        self.tanh = nn.Tanh()

    def forward(self,x):
        n1 = self.n1(x)
        n2 = self.n2(F.leaky_relu(n1, 0.2))
        n3 = self.n3(F.leaky_relu(n2, 0.2))
        n4 = self.n4(F.leaky_relu(n3, 0.2))
        n5 = self.n5(F.leaky_relu(n4, 0.2))
        n6 = self.n6(F.leaky_relu(n5, 0.2))
        n7 = self.n7(F.leaky_relu(n6, 0.2))
        n8 = self.n8(F.leaky_relu(n7, 0.2))
        m1 = torch.cat([F.dropout(self.m1(F.relu(n8)), 0.5, training=True), n7], 1)
        m2 = torch.cat([F.dropout(self.m2(F.relu(m1)), 0.5, training=True), n6], 1)
        m3 = torch.cat([F.dropout(self.m3(F.relu(m2)), 0.5, training=True), n5], 1)
        m4 = torch.cat([self.m4(F.relu(m3)), n4], 1)
        m5 = torch.cat([self.m5(F.relu(m4)), n3], 1)
        m6 = torch.cat([self.m6(F.relu(m5)), n2], 1)
        m7 = torch.cat([self.m7(F.relu(m6)), n1], 1)
        m8 = self.m8(F.relu(m7))

        return self.tanh(m8)

In [ ]:
dim_d = 64

# Discriminator class designed for taking input of shape (256,256,3)
class Disc(nn.Module):
    def __init__(self, inst_norm=False):
        super(Disc,self).__init__()
        self.c1 = conv(dim_c*2, dim_d, 4, 2, 1)
        self.c2 = conv_n(dim_d, dim_d*2, 4, 2, 1, inst_norm=inst_norm)
        self.c3 = conv_n(dim_d*2, dim_d*4, 4, 2, 1, inst_norm=inst_norm)
        self.c4 = conv_n(dim_d*4, dim_d*8, 4, 1, 1, inst_norm=inst_norm)
        self.c5 = conv(dim_d*8, 1, 4, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, y):
        xy=torch.cat([x,y],dim=1)
        xy=F.leaky_relu(self.c1(xy), 0.2)
        xy=F.leaky_relu(self.c2(xy), 0.2)
        xy=F.leaky_relu(self.c3(xy), 0.2)
        xy=F.leaky_relu(self.c4(xy), 0.2)
        xy=self.c5(xy)

        return self.sigmoid(xy)

def weights_init(z):
    cls_name =z.__class__.__name__
    if cls_name.find('Conv')!=-1 or cls_name.find('Linear')!=-1:
        nn.init.normal_(z.weight.data, 0.0, 0.02)
        nn.init.constant_(z.bias.data, 0)
    elif cls_name.find('BatchNorm')!=-1:
        nn.init.normal_(z.weight.data, 1.0, 0.02)
        nn.init.constant_(z.bias.data, 0)

In [ ]:
BCE = nn.BCELoss() #binary cross-entropy
L1 = nn.L1Loss() # L1 loss
L2=nn.MSELoss() # L2 loss
#instance normalization
Gen_model = Gen(inst_norm).to(device)
Disc = Disc(inst_norm).to(device)
generator = Gen(inst_norm).to(device)
#optimizers
# Gen_optim = optim.Adam(Gen.parameters(), lr=2e-4, betas=(0.5, 0.999), weight_decay=0.35)
Gen_optim = optim.Adam(Gen_model.parameters(), lr=2e-4, betas=(0.5, 0.999))   #lr=1e-4
Disc_optim = optim.Adam(Disc.parameters(), lr=2e-4, betas=(0.5, 0.999))   # lr=5e-5

In [ ]:
# training and displaying the results of training on few training samples


iter_per_plot = 10
epochs = 10
L1_lambda = 100.0


for ep in range(epochs):
    for i, data in enumerate(load_Train):
        start_time = time.time()
        size = data.shape[0]
        

        x,y = split(data.to(device))
        ## for t1 generation using t2 
        # y, x= split(data.to(device))

        

        r_masks = torch.ones(size,1,30,30).to(device)
        f_masks = torch.zeros(size,1,30,30).to(device)

        # disc
        Disc.zero_grad()
        r_patch=Disc(y,x)
       
        
        
        r_disc_loss=L2(r_patch,r_masks)

        fake=Gen_model(x)
      

        #fake_patch
        f_patch = Disc(fake.detach(),x)
        # print(f_patch.shape,f_masks.shape)
        
        f_disc_loss=L2(f_patch,f_masks)
        Disc_loss = r_disc_loss + f_disc_loss
        
        Disc_loss.backward()
        Disc_optim.step()

        # gen
        Gen_model.zero_grad()
        f_patch = Disc(fake,x)
        f_gan_loss=L2(f_patch,r_masks)

        L1_loss = L1(fake,y)
        Gen_loss = f_gan_loss + L1_lambda*L1_loss
        
        Gen_loss.backward()

        Gen_optim.step()
        end_time = time.time()  # End measuring time for each iteration
        elapsed_time = end_time - start_time  # Calculate elapsed time
       

        
        if (i+1)%iter_per_plot == 0 :

            print('Epoch [{}/{}], Step [{}/{}], disc_loss: {:.4f}, gen_loss: {:.4f},Disc(real): {:.2f}, Disc(fake):{:.2f}, gen_loss_gan:{:.4f}, gen_loss_L1:{:.4f}'.format(ep+1, epochs, i+1, len(load_Train), Disc_loss.item(), Gen_loss.item(), r_disc_loss.item(), f_disc_loss.item(), f_gan_loss.item(), L1_loss.item()))

    

    if (ep+1)%5==0:
    
        with torch.no_grad():
            Gen_model.eval()
            for data in load_Train:
                # print(data.shape)
                x, y = split(data.to(device))

                fake = Gen_model(x)
                    
                for j in range(fake.shape[0]):
                    if j%10==0:
                        figs=plt.figure(figsize=(10,10))
            
                        plt.subplot(1,3,1)
                        plt.axis("off")
                        plt.title("input image")
                        plt.imshow(np.transpose(vutils.make_grid(x[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
                    
                        plt.subplot(1,3,2)
                        plt.axis("off")
                        plt.title("generated image")
                        plt.imshow(np.transpose(vutils.make_grid(fake[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
                    
                        plt.subplot(1,3,3)
                        plt.axis("off")
                        plt.title("ground truth")
                        plt.imshow(np.transpose(vutils.make_grid(y[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
        
                break
            Gen_model.train()
            
    if ep+1==epochs:
        torch.save(Gen_model.state_dict(), 't1_to_t2_model_patch.pth')  # saving the model after training
            


In [ ]:
####visualsing the generated image and plotting the error map

batch_size = 1  # Set the batch size to 1 to get a single image in each iteration
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
c=0
generator.load_state_dict(torch.load('t1_to_t2_model_augmented.pth', map_location=device)) # loading best saved model

for i,data in enumerate(test_loader):
    if i%10==0:
        c+=1
        t_batch=data['image1']
        t_x, t_y = split(t_batch)

        ## for t1 generation using t2 
        # t_y, t_x= split(t_batch.to(device))
        
    
        
        
        with torch.no_grad():
            generator.eval()
            fk= generator(t_x.to(device))
            t_y=t_y.to(device)
            figs=plt.figure(figsize=(10,10))

            plt.subplot(1,4,1)
            plt.axis("off")
            plt.title("input image")
            
            plt.imshow((np.transpose(vutils.make_grid(t_x[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0))))
            
            
            plt.subplot(1,4,2)
            plt.axis("off")
            plt.title("generated image")
            plt.imshow((np.transpose(vutils.make_grid(fk[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0))))
            # plt.colorbar()
            
            plt.subplot(1,4,3)
            plt.axis("off")
            plt.title("ground truth")
            plt.imshow((np.transpose(vutils.make_grid(t_y[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0))))
            
            plt.subplot(1,4,4)
            plt.axis("off")
            plt.title("error map")
    
            fake_img=fake_img.to(device)
            data_eg=np.transpose(np.abs((y_img[0]-fake_img[0]).cpu().numpy()),(1,2,0))
            normalized_data=Normalize()(data_eg)
            normalized_data=np.mean(normalized_data, axis=2, keepdims=True)
            # print(normalized_data.shape)
            plt.imshow(np.rot90((normalized_data),k=-1),cmap='jet')


            if c==20:
                break
                

In [ ]:

##plotting histogram


c=0
for i,data in enumerate (test_loader):
    if i%155==0:
        c+=1
        t_batch=data['image1']
        t_x, t_y = split(t_batch)
        ## for t1 generation using t2 
        # t_y, t_x= split(t_batch.to(device))
        
        with torch.no_grad():
            generator.eval()
            fk= generator(t_x.to(device))
            t_y=t_y.to(device)
        figs=plt.figure(figsize=(12,3))
        plt.subplot(1,2,1)
        fake_np_rgb=np.transpose((fk[0].cpu().numpy()),(1,2,0))
        fake_np_rgb=Normalize()(fake_np_rgb)
        fake_np_gray=np.mean(fake_np_rgb, axis=2)

# Plot the histogram
        plt.hist(fake_np_gray.flatten(), bins=256, range=[0, 1], density=True, color='red', alpha=0.5)
        plt.yscale('log')
        plt.xlabel('Intensity Value')
        plt.ylabel('Frequency')
        plt.title('Histogram of Grayscale Generated Image (Intensity [0, 1])')
        # plt.show()
        # figs=plt.figure(figsize=(10,10))
        plt.subplot(1,2,2)
        true_np_rgb=np.transpose((t_y[0].cpu().numpy()),(1,2,0))
        true_np_rgb=Normalize()(true_np_rgb)
        true_np_gray=np.mean(true_np_rgb, axis=2)
# Plot the histogram
        plt.hist(true_np_gray.flatten(), bins=256, range=[0, 1], density=True, color='gray', alpha=0.5)
        plt.yscale('log')
        plt.xlabel('Intensity Value')
        plt.ylabel('Frequency')
        plt.title('Histogram of Grayscale True Image (Intensity [0, 1])')
        plt.show()

        if c==10:
            break


In [ ]:


# calculating mean rmse and mean ssim
# # Assuming you have a dataset called 'test_dataset'
batch_size = 1  # Set the batch size to 1 to get a single image in each iteration
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
error_list=[]
structural_similarity=[]
# Now you can iterate over it with random samples
for i,data in enumerate (test_loader):
    t_batch=data['image1']
    t_x, t_y = split(t_batch)
    ## for t1 generation using t2 
    # t_y, t_x= split(t_batch.to(device))
    with torch.no_grad():
        generator.eval()
        fk_batch = generator(t_x.to(device))
    
    t_y_numpy=t_y.numpy()
    fk_batch_numpy=fk_batch.cpu().detach().numpy()
   
    error_list.append(np.sqrt(np.sum((t_y_numpy-fk_batch_numpy)**2)))
    t_y_numpy=np.transpose(t_y_numpy.squeeze(), (1, 2, 0))
    fk_batch_numpy=np.transpose(fk_batch_numpy.squeeze(), (1, 2, 0))
    
    ssi=ssim(t_y_numpy,fk_batch_numpy,multichannel=True,win_size=3,data_range=2.0)
    structural_similarity.append(ssi)
    
error_list=np.array(error_list)
structural_similarity_np=np.array(structural_similarity)

print('mean rmse error :',np.mean(error_list))
print('max rmse error:',np.max(error_list))
print('min rmse error:',np.min(error_list))
print('mean ssim :',np.mean(structural_similarity_np))
print('min ssim :',np.min(structural_similarity_np))
print('max ssim :',np.max(structural_similarity_np))


In [ ]:
# function to calculate cnr given tumor mask
def calculate_cnr(image, tumor_mask):
      # Calculate signal difference
    if np.any(tumor_mask == 1):
        tumor_intensity= np.mean(image[tumor_mask == 1])
    else:
        tumor_intensity = np.nan
        
    if np.any(tumor_mask == 1):
        background_intensity=np.mean(image[tumor_mask == 0])
        noise = np.std(image[tumor_mask == 0])
    else:
        background_intensity = np.nan
        noise=np.nan

    if np.isnan(tumor_intensity) or np.isnan(background_intensity):
        # Set CNR to a specific value or return NaN
        cnr = 0 # or cnr = 0
    else:
        signal_difference = np.abs(tumor_intensity - background_intensity)
        # print(signal_difference)
        # Estimate noise (standard deviation of background noise)
        
        # print(noise)
        if noise==0 or np.isnan(noise):
            cnr=0
        else:
        # Calculate CNR
            cnr = signal_difference / noise
    
    # Calculate CNR
        # cnr = signal_difference / noise
    
    return cnr


In [ ]:
# calculating relative cnr
relative_cnr=[]
for i,data in enumerate (test_loader):
    # if i==10:
    # print(i)
    t_batch=data['image1']
    mask=data['image2'].numpy().squeeze()
    # print((mask.shape))
    _, binary_mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)
    # print(np.unique(binary_mask))
    
    t_x, t_y = split(t_batch)
    with torch.no_grad():
        generator.eval()
        fk_batch = generator(t_x.to(device))
    t_y_numpy=t_y.numpy()
    fk_batch_numpy=fk_batch.cpu().detach().numpy()
    t_y_numpy=np.transpose(t_y_numpy.squeeze(), (1, 2, 0))
    fk_batch_numpy=np.transpose(fk_batch_numpy.squeeze(), (1, 2, 0))
    
    cnr_fake=calculate_cnr(fk_batch_numpy, binary_mask)
    cnr_real=calculate_cnr(t_y_numpy, binary_mask)

    if cnr_fake!=0 and cnr_real!=0:
        
        relative_cnr.append(np.abs(cnr_fake-cnr_real))
    
relative_cnr_numpy=np.array(relative_cnr)
print(np.mean(relative_cnr_numpy))
        
    